In [24]:
import random

def generate_number(integer=True, digits=1):
    if integer:
        return random.randint(10**(digits-1), 10**digits - 1)
    else:
        return random.uniform(10**(digits-1), 10**digits - 1)

def generate_addition(integer=True, digits=1):
    a = generate_number(integer, digits)
    b = generate_number(integer, digits)
    if integer:
        question = f"{a} + {b} ="
        answer = a + b
    else:
        question = f"{a:.1f} + {b:.1f} ="
        answer = a + b
    return question, answer

def generate_subtraction(integer=True, digits=1):
    a = generate_number(integer, digits)
    b = generate_number(integer, digits)
    if integer:
        question = f"{a} - {b} ="
        answer = a - b
    else:
        question = f"{a:.1f} - {b:.1f} ="
        answer = a - b
    return question, answer

def generate_multiplication(integer=True, digits=1):
    a = generate_number(integer, digits)
    b = generate_number(integer, digits)
    if integer:
        question = f"{a} * {b} ="
        answer = a * b
    else:
        question = f"{a:.1f} * {b:.1f} ="
        answer = a * b
    return question, answer

def generate_division(integer=True, digits=1):
    b = generate_number(integer, digits)
    a = b * generate_number(integer, digits)
    if integer:
        question = f"{a} / {b} ="
        answer = a / b
    else:
        question = f"{a:.1f} / {b:.1f} ="
        answer = a / b
    return question, answer

def generate_mixed_operation(integer=True, num_terms=3, digits=1):
    operations = ['+', '-', '*', '/']
    terms = [generate_number(integer, digits) for _ in range(num_terms)]
    chosen_operations = random.choices(operations, k=num_terms-1)
    if integer:
        question = " ".join(f"{terms[i]} {chosen_operations[i]}" for i in range(num_terms-1)) + f" {terms[-1]} ="
        answer = terms[0]
        for i in range(num_terms-1):
            if chosen_operations[i] == '+':
                answer += terms[i+1]
            elif chosen_operations[i] == '-':
                answer -= terms[i+1]
            elif chosen_operations[i] == '*':
                answer *= terms[i+1]
            elif chosen_operations[i] == '/':
                answer /= terms[i+1]
    else:
        question = " ".join(f"{terms[i]:.1f} {chosen_operations[i]}" for i in range(num_terms-1)) + f" {terms[-1]:.1f} ="
        answer = terms[0]
        for i in range(num_terms-1):
            if chosen_operations[i] == '+':
                answer += terms[i+1]
            elif chosen_operations[i] == '-':
                answer -= terms[i+1]
            elif chosen_operations[i] == '*':
                answer *= terms[i+1]
            elif chosen_operations[i] == '/':
                answer /= terms[i+1]

    return question, answer

def generate_problems(num_problems):
    problems = []
    answers = []

    for _ in range(num_problems):
        problem_type = random.choice(['addition', 'subtraction', 'multiplication', 'division', 'mixed'])
        integer = random.choice([True, False])
        num_terms = random.randint(2, 5)  # 2から5の項数をランダムに選択
        digits = random.randint(1, 5)  # 1から5桁をランダムに選択
        
        if problem_type == 'addition':
            q, a = generate_addition(integer, digits)
        elif problem_type == 'subtraction':
            q, a = generate_subtraction(integer, digits)
        elif problem_type == 'multiplication':
            q, a = generate_multiplication(integer, digits)
        elif problem_type == 'division':
            q, a = generate_division(integer, digits)
        elif problem_type == 'mixed':
            q, a = generate_mixed_operation(integer, num_terms, digits)

        problems.append(q)
        answers.append(f"{q} {a:.1f}" if not integer else f"{q} {a}")

    return problems, answers




In [35]:
from tqdm import tqdm

q_list = []
a_list=[]
n_problems=10**6

for i in tqdm(range(n_problems)):
    num_problems = random.randint(1,20)  # 任意の問題数に調整可能
    problems, answers = generate_problems(num_problems)
    problems = "\n".join(problems)
    answers = "\n".join(answers)
    q_list.append(problems)
    a_list.append(answers)

  0%|          | 0/1000000 [00:00<?, ?it/s]

100%|██████████| 1000000/1000000 [00:40<00:00, 24439.62it/s]


In [39]:
import pandas as pd
df = pd.DataFrame({'question': q_list, 'answer': a_list})
df.to_parquet("data/sansu.parquet")

In [41]:
from huggingface_hub import HfApi, logging
hf = HfApi()
hf.upload_file(path_or_fileobj="data/sansu.parquet",
                path_in_repo=f"1.parquet",
                repo_id="kanhatakeyama/Sansu", repo_type="dataset")




/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
sansu.parquet: 100%|██████████| 317M/317M [00:24<00:00, 13.0MB/s]    


CommitInfo(commit_url='https://huggingface.co/datasets/kanhatakeyama/Sansu/commit/2443d64a775ca9aed5eb0102aab84e18ea860487', commit_message='Upload 1.parquet with huggingface_hub', commit_description='', oid='2443d64a775ca9aed5eb0102aab84e18ea860487', pr_url=None, pr_revision=None, pr_num=None)